<a href="https://colab.research.google.com/github/HumbertoOMendess/HumbertoOMendess/blob/main/TrabalhoAG_HumbertoMendes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports necessários

In [157]:
import numpy as np
from random import randint, uniform
import matplotlib.pyplot as plt

# Classe do algoritmo genético

In [158]:
class AG(object):
    def __init__(self, limitantes, tamanho_populacao_inicial, numero_geracoes, taxa_mutacao, taxacrossover, numero_bits=10): #Inicialização dos valores que serão utilizado durante o trabalho.
        self.limite_inferior = limitantes[0]
        self.limite_superior = limitantes[1]
        self.numero_geracoes = numero_geracoes
        self.taxa_mutacao = taxa_mutacao
        self.taxacrossover = taxacrossover
        self.numero_bits = numero_bits
        self.tamanho_populacao_inicial = tamanho_populacao_inicial
        self.populacao = self.geracao_populacao_inicial()

    def geracao_populacao_inicial(self): #Função responsavel por gerar a população inicial.
        cromossomos = []
        for _ in range(self.tamanho_populacao_inicial):
            cromossomos.append(np.random.randint(2, size=self.numero_bits))
        return cromossomos
    
    @staticmethod
    def converter_binario(cromossomo): #Função responsavel por converter os numeros.
        s = "0b"
        for bit in cromossomo:
            s += str(bit)
        return int(s, 2)
  
    def mapear_cromossomo(self, cromossomo): #Função responsavel por mapear um valor entre -10 e 10.
        return self.limite_inferior + (self.limite_superior - self.limite_inferior) * self.converter_binario(cromossomo) / (2 ** self.numero_bits - 1)

    @staticmethod
    def funcao_aptidao(x): #Função de aptidão para calcular X.
        return x * x - 3 * x + 4
    
    def torneio(self): #Função responsavel por realizar o torneio que seleciona aleatoriamente dois cromossomos da população e seleciona o mais apto.
        melhor_cromossomo = None
        for _ in range(2):
            pos_cromossomo = randint(0, len(self.populacao) - 1)
            cromossomo_desafiante = self.populacao[pos_cromossomo]
            if melhor_cromossomo is None or self.funcao_aptidao(self.mapear_cromossomo(melhor_cromossomo)) < self.funcao_aptidao(self.mapear_cromossomo(cromossomo_desafiante)):
                melhor_cromossomo = cromossomo_desafiante
        return melhor_cromossomo

    def crossover(self, pai_1, pai_2): #Função responsavel por receber dois pais e gerar dois filhos.
        filho_1 = np.copy(pai_1)
        filho_2 = np.copy(pai_2)
        probabilidade = uniform(0, 1)
        if probabilidade <= self.taxacrossover:
            pos_recorte_1 = randint(0, self.numero_bits - 1)
            pos_recorte_2 = randint(pos_recorte_1 + 1, self.numero_bits)
            filho_1[pos_recorte_1:pos_recorte_2] = pai_2[pos_recorte_1:pos_recorte_2]
            filho_2[pos_recorte_1:pos_recorte_2] = pai_1[pos_recorte_1:pos_recorte_2]
        return filho_1, filho_2
    
    def mutacao(self, cromossomo): #Função responsável por realizar a mutação de um cromossomo.
        for i in range(self.numero_bits):
            probabilidade = uniform(0, 1)
            if probabilidade <= self.taxa_mutacao:
                cromossomo[i] = 1 - cromossomo[i]
    
    def gerar_pontos(self, lista_pontos): #Função necessária para gerar pontos em relaçao a seleção do melhor cromossomo.
        tam_lista = len(lista_pontos)
        vetor = np.empty(tam_lista)
        for i in range(tam_lista):
            vetor[i] = self.mapear_cromossomo(lista_pontos[i])
        return vetor
    
    def selecionar_melhor(self): #Função reponsavel por seleciona o melhor cromossomo na população.
        vetor_pontos = self.gerar_pontos(self.populacao)
        pos_melhor = int(np.argmax(self.funcao_aptidao(vetor_pontos)))
        melhor_binario = self.populacao[pos_melhor]
        return vetor_pontos[pos_melhor], melhor_binario

    def selecionar(self): #Função responsavel por gerar novas populaçoes, e por chamar funçoes para calcular crossover, mutação, torneio.
        melhor_cromossomo, _ = self.selecionar_melhor()
        print("Melho indivíduo da geração 1: x= {:.5f}".format(melhor_cromossomo))
        historico = [self.populacao.copy()]
        for geracao in range(self.numero_geracoes):
            prox_geracao = []
            while len(prox_geracao) < self.tamanho_populacao_inicial:
                pai_1 = self.torneio()
                pai_2 = self.torneio()
                filho_1, filho_2 = self.crossover(pai_1, pai_2)
                self.mutacao(filho_1)
                self.mutacao(filho_2)
                prox_geracao.append(filho_1)
                prox_geracao.append(filho_2)
            melhor_cromossomo, melhor_binario = self.selecionar_melhor()
            prox_geracao.append(melhor_binario)
            if (geracao + 1) % 1 == 0:
                print("Melho indivíduo da geração {:d}: x= {:.5f}".format(geracao + 1, melhor_cromossomo))
                historico.append(prox_geracao.copy())
            self.populacao = prox_geracao

## Função main para realizar os testes

In [159]:
def main():
    algoritmo_genetico = AG([-10, 10], 30, 50, 0.01, 0.7, 10) #Passagem de parametros para a função init.
    algoritmo_genetico.selecionar() #Chamada da função para selecionar o melhor individuo de cada geração.
    

if __name__ == '__main__':
    main()

Melho indivíduo da geração 1: x= -8.24047
Melho indivíduo da geração 1: x= -8.24047
Melho indivíduo da geração 2: x= -8.24047
Melho indivíduo da geração 3: x= -9.90225
Melho indivíduo da geração 4: x= -9.96090
Melho indivíduo da geração 5: x= -9.96090
Melho indivíduo da geração 6: x= -9.96090
Melho indivíduo da geração 7: x= -9.96090
Melho indivíduo da geração 8: x= -10.00000
Melho indivíduo da geração 9: x= -10.00000
Melho indivíduo da geração 10: x= -10.00000
Melho indivíduo da geração 11: x= -10.00000
Melho indivíduo da geração 12: x= -10.00000
Melho indivíduo da geração 13: x= -10.00000
Melho indivíduo da geração 14: x= -10.00000
Melho indivíduo da geração 15: x= -10.00000
Melho indivíduo da geração 16: x= -10.00000
Melho indivíduo da geração 17: x= -10.00000
Melho indivíduo da geração 18: x= -10.00000
Melho indivíduo da geração 19: x= -10.00000
Melho indivíduo da geração 20: x= -10.00000
Melho indivíduo da geração 21: x= -10.00000
Melho indivíduo da geração 22: x= -10.00000
Melho 